### 1.baseline与coco-model的图片比较（均与原始text比较）
#### 1.1 组间相似度

In [12]:
import json
img_text_dic = {}
with open("/home/cl/CV_Proj/evaluate/metadata_test.jsonl") as fin:
    for line in fin.readlines():
        tmp_dic = json.loads(line)
        img_text_dic[tmp_dic["file_name"]] = tmp_dic["text"]
#         print(tmp_dic["file_name"],tmp_dic["text"])

In [13]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import clip


In [14]:
model, preprocess = clip.load('ViT-B/32')

model.cuda().eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [15]:
ori_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/original_imgs_5003'
base_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/baseline_inference'
coco_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model_inference'

bt_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model-bt_inference'
crop_img_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-crop_swap_inference'
b1n10_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta1_n10_inference'

b1n5_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model-linear_p_beta1_n5_inference'
b4n5_img_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta4_n5_inference'
b8n5_img_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta8_n5_inference'

In [27]:
# encode图片
img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/original_imgs_5003'
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding origin images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
#encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

# 计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between original images and text is:",img_text_sim.item())

Encoding origin images: 100%|██████████████████████████████████████████████████████████████████████████| 5003/5003 [00:42<00:00, 116.42it/s]


The similarity between original images and text is: 0.15154661238193512


In [28]:
#encode图片
img_path = base_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
# #encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

# #计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between baseline model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:30<00:00, 162.32it/s]


The similarity between baseline model generated images and text is: 0.1415233016014099


In [29]:
#encode图片
img_path = coco_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
# #encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list)
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input.cuda())

#     text_features.append(text_feature)
    


# #计算相似度

# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between coco model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:36<00:00, 137.29it/s]


The similarity between coco model generated images and text is: 0.1465878039598465


In [19]:
#encode图片
# img_path = crop_img_path
# images = []
# text = []

# for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
#     cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
#     cur_image = preprocess(cur_image)
#     images.append(cur_image)
    
#     cur_text = [img_text_dic[item]]
#     text.append(cur_text)

# img_inputs = torch.stack(images, dim=0).cuda()

# with torch.no_grad():
#     img_features = model.encode_image(img_inputs)
    
# #encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

# #计算相似度
# text_features = torch.cat(text_features, dim=0)

with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
    print("The similarity between corp_swap model generated images and text is:",img_text_sim.item())

The similarity between corp_swap model generated images and text is: 0.14705707132816315


In [20]:
#encode图片
img_path = bt_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
# #encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

#计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between bt model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:42<00:00, 118.59it/s]


The similarity between bt model generated images and text is: 0.1469103842973709


In [25]:
#encode图片
img_path = b1n10_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
#encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

#计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between b1n10 model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:35<00:00, 140.05it/s]


The similarity between b1n10 model generated images and text is: 0.14710772037506104


In [22]:
#encode图片
img_path = b1n5_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
# #encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

#计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between b1n5 model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:44<00:00, 112.43it/s]


The similarity between b1n5 model generated images and text is: 0.1466483622789383


In [23]:
#encode图片
img_path = b4n5_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
#encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

#计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between b4n5 model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:30<00:00, 162.60it/s]


The similarity between b4n5 model generated images and text is: 0.1467028707265854


In [24]:
#encode图片
img_path = b8n5_img_path
images = []
text = []

for i, item in enumerate(tqdm(img_text_dic,desc='Encoding images')):
    cur_image = Image.open(os.path.join(img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

img_inputs = torch.stack(images, dim=0).cuda()

with torch.no_grad():
    img_features = model.encode_image(img_inputs)
    
#encode文本
# text_features = []
# for idx, sent_list in enumerate(tqdm(text, desc='Encoding sentences')):
#     try:
#         text_input = clip.tokenize(sent_list).cuda()
#     except Exception as e:
#         print(str(e))
#         continue

#     with torch.no_grad():
#         text_feature = model.encode_text(text_input)

#     text_features.append(text_feature)

#计算相似度
# text_features = torch.cat(text_features, dim=0)
with torch.no_grad():
    img_text_sim = F.cosine_similarity(text_features.unsqueeze(1).cpu().float(), img_features.unsqueeze(0).cpu().float(), dim=-1).mean()
print("The similarity between b8n5 model generated images and text is:",img_text_sim.item())

Encoding images: 100%|█████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:36<00:00, 137.80it/s]


The similarity between b8n5 model generated images and text is: 0.14661511778831482
